In [1]:
import pandas as pd
import h2o
h2o.init()

dt = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

valid_ids = dt.policy_id.sample(frac=0.25)
train_dt = dt[~dt.policy_id.isin(valid_ids)]
valid_dt = dt[dt.policy_id.isin(valid_ids)]
train_dt = train_dt.reset_index(drop=True)
valid_dt = valid_dt.reset_index(drop=True)

train_dt_hex = h2o.H2OFrame(train_dt)
valid_dt_hex = h2o.H2OFrame(valid_dt)
test_hex = h2o.H2OFrame(test)

train_dt_hex['make'] = train_dt_hex['make'].asfactor()
valid_dt_hex['make'] = valid_dt_hex['make'].asfactor()
test_hex['make'] = test_hex['make'].asfactor()

train_dt_hex['is_claim'] = train_dt_hex['is_claim'].asfactor()
valid_dt_hex['is_claim'] = valid_dt_hex['is_claim'].asfactor()

selvars = ['policy_tenure','age_of_car','age_of_policyholder','population_density','airbags','width','height','ncap_rating',
    'area_cluster','segment','model','fuel_type','max_torque','max_power','engine_type','is_esc','is_adjustable_steering','is_parking_sensors','is_parking_camera','rear_brakes_type','transmission_type',
'steering_type','is_front_fog_lights','is_rear_window_wiper','is_rear_window_defogger','is_brake_assist','is_power_door_locks','is_central_locking','is_power_steering','is_driver_seat_height_adjustable']

numvars = ['policy_tenure','age_of_car','age_of_policyholder','population_density','airbags','displacement','cylinder','gear_box','turning_radius','length','width','height','gross_weight','ncap_rating']
catvars = ['area_cluster','segment','model','fuel_type','max_torque','max_power','engine_type','is_esc','is_adjustable_steering','is_tpms','is_parking_sensors','is_parking_camera','rear_brakes_type','transmission_type',
'steering_type','is_front_fog_lights','is_rear_window_wiper','is_rear_window_washer','is_rear_window_defogger','is_brake_assist','is_power_door_locks','is_central_locking','is_power_steering','is_driver_seat_height_adjustable',
'is_day_night_rear_view_mirror','is_ecw','is_speed_alert']

allvars =  ['policy_tenure','age_of_car','age_of_policyholder','population_density','airbags','displacement','cylinder','gear_box','turning_radius','length','width','height','gross_weight','ncap_rating',
'area_cluster','segment','model','fuel_type','max_torque','max_power','engine_type','is_esc','is_adjustable_steering','is_tpms','is_parking_sensors','is_parking_camera','rear_brakes_type','transmission_type',
'steering_type','is_front_fog_lights','is_rear_window_wiper','is_rear_window_washer','is_rear_window_defogger','is_brake_assist','is_power_door_locks','is_central_locking','is_power_steering','is_driver_seat_height_adjustable',
'is_day_night_rear_view_mirror','is_ecw','is_speed_alert']

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,4 hours 17 mins
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,16 days
H2O_cluster_name:,H2O_from_python_Paul_4kgr44
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.946 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [2]:
# glm model
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.estimators import H2ORandomForestEstimator
othervars = [i for i in allvars if i not in selvars]
mdl_glm = H2OGeneralizedLinearEstimator(family='binomial',lambda_search=True,nfolds=5).train(x=selvars,y='is_claim',training_frame=train_dt_hex,validation_frame=valid_dt_hex)
mdl_gbm = H2OGradientBoostingEstimator(ntrees=65,min_rows=150,max_depth=3,nfolds=5).train(x=selvars,y='is_claim',training_frame=train_dt_hex,validation_frame=valid_dt_hex)
mdl_rf = H2ORandomForestEstimator(ntrees=160,min_rows=50,max_depth=6,nfolds=5).train(x=selvars,y='is_claim',training_frame=train_dt_hex,validation_frame=valid_dt_hex)

selvars_cat = [i for i in catvars if i in selvars]
from catboost import CatBoostClassifier
clf = CatBoostClassifier(iterations=25,depth=7).fit(
    train_dt[selvars], train_dt.is_claim,
    cat_features=selvars_cat,
    eval_set=(valid_dt[selvars], valid_dt.is_claim),
)

pred_dt = h2o.as_list(mdl_glm.predict(train_dt_hex),use_pandas=True)
train_dt['mdl1_glm_score'] = pred_dt['p1']
pred_dt = h2o.as_list(mdl_gbm.predict(train_dt_hex),use_pandas=True)
train_dt['mdl1_gbm_score'] = pred_dt['p1']
pred_dt = h2o.as_list(mdl_rf.predict(train_dt_hex),use_pandas=True)
train_dt['mdl1_rf_score'] = pred_dt['p1']
pred_val = clf.predict_proba(train_dt[selvars])
train_dt['mdl1_catboost_score'] = [pred_val[val][1] for val in range(len(pred_val))]

pred_dt = h2o.as_list(mdl_glm.predict(valid_dt_hex),use_pandas=True)
valid_dt['mdl1_glm_score'] = pred_dt['p1']
pred_dt = h2o.as_list(mdl_gbm.predict(valid_dt_hex),use_pandas=True)
valid_dt['mdl1_gbm_score'] = pred_dt['p1']
pred_dt = h2o.as_list(mdl_rf.predict(valid_dt_hex),use_pandas=True)
valid_dt['mdl1_rf_score'] = pred_dt['p1']
pred_val = clf.predict_proba(valid_dt[selvars])
valid_dt['mdl1_catboost_score'] = [pred_val[val][1] for val in range(len(pred_val))]

pred_dt = h2o.as_list(mdl_glm.predict(test_hex),use_pandas=True)
test['mdl1_glm_score'] = pred_dt['p1']
pred_dt = h2o.as_list(mdl_gbm.predict(test_hex),use_pandas=True)
test['mdl1_gbm_score'] = pred_dt['p1']
pred_dt = h2o.as_list(mdl_rf.predict(test_hex),use_pandas=True)
test['mdl1_rf_score'] = pred_dt['p1']
pred_val = clf.predict_proba(test[selvars])
test['mdl1_catboost_score'] = [pred_val[val][1] for val in range(len(pred_val))]

glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Learning rate set to 0.401422
0:	learn: 0.3601710	test: 0.3596923	best: 0.3596923 (0)	total: 238ms	remaining: 5.7s
1:	learn: 0.2694577	test: 0.2692912	best: 0.2692912 (1)	total: 319ms	remaining: 3.67s
2:	learn: 0.2434306	test: 0.2429648	best: 0.2429648 (2)	total: 375ms	remaining: 2.75s
3:	learn: 0.2349176	test: 0.2344204	best: 0.2344204 (3)	total: 444ms	remaining: 2.33s
4:	learn: 0.2305794	test: 0.2302244	best: 0.2302244 (4)	total: 514ms	remaining: 2.06s
5:	learn: 0.2296488	test: 0.2291888	best: 0.2291888 (5)	total: 559ms	remaining: 1.77s
6:	learn: 0.2288040	test: 0.2285216	best: 0.2285216 (6)	total: 620ms	remaining: 1.59s
7:	learn: 0.2278347	test: 0.2280351	best: 0.2280351 (7)	total: 687ms	remaining: 1.46

In [4]:
score_vars_mdls = ['mdl1_glm_score','mdl1_gbm_score','mdl1_rf_score','mdl1_catboost_score']
train_dt_hex = h2o.H2OFrame(train_dt)
valid_dt_hex = h2o.H2OFrame(valid_dt)
test_hex = h2o.H2OFrame(test)

train_dt_hex['make'] = train_dt_hex['make'].asfactor()
valid_dt_hex['make'] = valid_dt_hex['make'].asfactor()
test_hex['make'] = test_hex['make'].asfactor()

train_dt_hex['is_claim'] = train_dt_hex['is_claim'].asfactor()
valid_dt_hex['is_claim'] = valid_dt_hex['is_claim'].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


In [8]:
mdl_glm_fnl = H2OGeneralizedLinearEstimator(family='binomial',lambda_search=True,interactions=score_vars_mdls,nfolds=5).train(x=score_vars_mdls,y='is_claim',training_frame=train_dt_hex,validation_frame=valid_dt_hex)

glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


In [11]:
import numpy as np
from sklearn.metrics import f1_score

thresh = 0.079

pred_dt = h2o.as_list(mdl_glm_fnl.predict(train_dt_hex),use_pandas=True)
train_dt['score'] = pred_dt['p1']
train_dt['prediction'] = np.where(train_dt.score>=thresh,1,0)

pred_dt = h2o.as_list(mdl_glm_fnl.predict(valid_dt_hex),use_pandas=True)
valid_dt['score'] = pred_dt['p1']
valid_dt['prediction'] = np.where(valid_dt.score>=thresh,1,0)

print(f1_score(y_true=train_dt.is_claim.values,y_pred=train_dt.prediction.values))
print(f1_score(y_true=valid_dt.is_claim.values,y_pred=valid_dt.prediction.values))

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
0.19164090848004303
0.17718940936863545


In [12]:
pred_dt = h2o.as_list(mdl_glm_fnl.predict(test_hex),use_pandas=True)
test['score'] = pred_dt['p1']
test['is_claim'] = np.where(test.score>=thresh,1,0)
print(test[['policy_id','is_claim']].head(5))
test[['policy_id','is_claim']].to_csv('output_iter5.csv',index=None)

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
  policy_id  is_claim
0   ID58593         1
1   ID58594         0
2   ID58595         0
3   ID58596         0
4   ID58597         0


In [13]:
othervars = [i for i in allvars if i not in othervars]
mdl2_glm = H2OGeneralizedLinearEstimator(family='binomial',lambda_search=True,nfolds=5).train(x=othervars,y='is_claim',training_frame=train_dt_hex,validation_frame=valid_dt_hex)
mdl2_gbm = H2OGradientBoostingEstimator(ntrees=65,min_rows=150,max_depth=3,nfolds=5).train(x=othervars,y='is_claim',training_frame=train_dt_hex,validation_frame=valid_dt_hex)
mdl2_rf = H2ORandomForestEstimator(ntrees=160,min_rows=50,max_depth=6,nfolds=5).train(x=othervars,y='is_claim',training_frame=train_dt_hex,validation_frame=valid_dt_hex)

pred_dt = h2o.as_list(mdl2_glm.predict(train_dt_hex),use_pandas=True)
train_dt['mdl2_glm_score'] = pred_dt['p1']
pred_dt = h2o.as_list(mdl2_gbm.predict(train_dt_hex),use_pandas=True)
train_dt['mdl2_gbm_score'] = pred_dt['p1']
pred_dt = h2o.as_list(mdl2_rf.predict(train_dt_hex),use_pandas=True)
train_dt['mdl2_rf_score'] = pred_dt['p1']

pred_dt = h2o.as_list(mdl2_glm.predict(valid_dt_hex),use_pandas=True)
valid_dt['mdl2_glm_score'] = pred_dt['p1']
pred_dt = h2o.as_list(mdl2_gbm.predict(valid_dt_hex),use_pandas=True)
valid_dt['mdl2_gbm_score'] = pred_dt['p1']
pred_dt = h2o.as_list(mdl2_rf.predict(valid_dt_hex),use_pandas=True)
valid_dt['mdl2_rf_score'] = pred_dt['p1']

pred_dt = h2o.as_list(mdl2_glm.predict(test_hex),use_pandas=True)
test['mdl2_glm_score'] = pred_dt['p1']
pred_dt = h2o.as_list(mdl2_gbm.predict(test_hex),use_pandas=True)
test['mdl2_gbm_score'] = pred_dt['p1']
pred_dt = h2o.as_list(mdl2_rf.predict(test_hex),use_pandas=True)
test['mdl2_rf_score'] = pred_dt['p1']

glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |████████████████████████

In [14]:
score_vars_mdls = ['mdl1_glm_score','mdl1_gbm_score','mdl1_rf_score','mdl1_catboost_score','mdl2_glm_score','mdl2_gbm_score','mdl2_rf_score']
train_dt_hex = h2o.H2OFrame(train_dt)
valid_dt_hex = h2o.H2OFrame(valid_dt)
test_hex = h2o.H2OFrame(test)

train_dt_hex['make'] = train_dt_hex['make'].asfactor()
valid_dt_hex['make'] = valid_dt_hex['make'].asfactor()
test_hex['make'] = test_hex['make'].asfactor()

train_dt_hex['is_claim'] = train_dt_hex['is_claim'].asfactor()
valid_dt_hex['is_claim'] = valid_dt_hex['is_claim'].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [16]:
mdl_glm_fnl = H2OGeneralizedLinearEstimator(family='binomial',lambda_search=True,interactions=score_vars_mdls,nfolds=5).train(x=score_vars_mdls,y='is_claim',training_frame=train_dt_hex,validation_frame=valid_dt_hex)

glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


In [18]:
thresh = 0.08

pred_dt = h2o.as_list(mdl_glm_fnl.predict(train_dt_hex),use_pandas=True)
train_dt['score'] = pred_dt['p1']

pred_dt = h2o.as_list(mdl_glm_fnl.predict(valid_dt_hex),use_pandas=True)
valid_dt['score'] = pred_dt['p1']

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
0.19226150767178118
0.17494949494949497


In [22]:
thresh = 0.09
train_dt['prediction'] = np.where(train_dt.score>=thresh,1,0)
valid_dt['prediction'] = np.where(valid_dt.score>=thresh,1,0)
print(f1_score(y_true=train_dt.is_claim.values,y_pred=train_dt.prediction.values))
print(f1_score(y_true=valid_dt.is_claim.values,y_pred=valid_dt.prediction.values))

0.19125459746628526
0.17635270541082163


In [23]:
pred_dt = h2o.as_list(mdl_glm_fnl.predict(test_hex),use_pandas=True)
test['score'] = pred_dt['p1']
test['is_claim'] = np.where(test.score>=thresh,1,0)
print(test[['policy_id','is_claim']].head(5))
test[['policy_id','is_claim']].to_csv('output_iter6.csv',index=None)

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
  policy_id  is_claim
0   ID58593         1
1   ID58594         0
2   ID58595         0
3   ID58596         0
4   ID58597         0


In [24]:
score_vars_mdls = ['mdl1_glm_score','mdl1_gbm_score','mdl1_rf_score','mdl1_catboost_score']
train_dt_hex = h2o.H2OFrame(train_dt)
valid_dt_hex = h2o.H2OFrame(valid_dt)
test_hex = h2o.H2OFrame(test)

train_dt_hex['make'] = train_dt_hex['make'].asfactor()
valid_dt_hex['make'] = valid_dt_hex['make'].asfactor()
test_hex['make'] = test_hex['make'].asfactor()

train_dt_hex['is_claim'] = train_dt_hex['is_claim'].asfactor()
valid_dt_hex['is_claim'] = valid_dt_hex['is_claim'].asfactor()

mdl_rf_fnl = H2ORandomForestEstimator(ntrees=15,min_rows=50,max_depth=3,nfolds=5).train(x=score_vars_mdls,y='is_claim',training_frame=train_dt_hex,validation_frame=valid_dt_hex)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


In [25]:
pred_dt = h2o.as_list(mdl_rf_fnl.predict(train_dt_hex),use_pandas=True)
train_dt['score'] = pred_dt['p1']
pred_dt = h2o.as_list(mdl_rf_fnl.predict(valid_dt_hex),use_pandas=True)
valid_dt['score'] = pred_dt['p1']

drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [34]:
thresh = 0.093
train_dt['prediction'] = np.where(train_dt.score>=thresh,1,0)
valid_dt['prediction'] = np.where(valid_dt.score>=thresh,1,0)
print(f1_score(y_true=train_dt.is_claim.values,y_pred=train_dt.prediction.values))
print(f1_score(y_true=valid_dt.is_claim.values,y_pred=valid_dt.prediction.values))

0.1925730676796651
0.17800047721307563


In [35]:
pred_dt = h2o.as_list(mdl_rf_fnl.predict(test_hex),use_pandas=True)
test['score'] = pred_dt['p1']
test['is_claim'] = np.where(test.score>=thresh,1,0)
print(test[['policy_id','is_claim']].head(5))
test[['policy_id','is_claim']].to_csv('output_iter7.csv',index=None)

drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
  policy_id  is_claim
0   ID58593         1
1   ID58594         0
2   ID58595         0
3   ID58596         0
4   ID58597         0


In [37]:
from h2o.automl import H2OAutoML
aml = H2OAutoML(max_models=100, seed=1,nfolds=5,max_runtime_secs=600)
aml.train(x=score_vars_mdls, y='is_claim', training_frame=train_dt_hex,leaderboard_frame=valid_dt_hex)

AutoML progress: |█
17:02:32.44: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_1_AutoML_2_20221113_170232


Status of Neuron Layers: predicting is_claim, 2-class classification, bernoulli distribution, CrossEntropy loss, 292 weights/biases, 8.8 KB, 457,251 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight           weight_rms           mean_bias              bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  --------------------  -------------------  ---------------------  -------------------
    1        4        Input      0.0
    2        10       Rectifier  0.0        0.0   0.0   0.002921305823838338   0.0015719132497906685  0.0         0.04288482589181512   0.41840362548828125  0.6893302002946634     0.26919639110565186
    3        10       Rectifier  0.0        0.0   0.0   0.0043203724460909145  0.006593452766537666   0.0         0.000738514323020354  0.28211212158203125  0.7810962365460451     0.14468914270401
    4        10       Rectifier  0.0        0.0   0.0   0.05267453717649914    0.16795068979263306    0.0         -0.04700528219458647  0.33212709426879883  0.9501382395469682     0.09783312678337097
    5        2        Softmax               0.0   0.0   0.014321174888755195   0.02818063646554947    0.0         -0.2477928575128317   1.3034982681274414   -0.008680960399484516  0.20770400762557983

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.06224797100616278
RMSE: 0.24949543283628015
LogLoss: 0.24111365330565512
Mean Per-Class Error: 0.3826735787604418
AUC: 0.6860058768723571
AUCPR: 0.1322355197088145
Gini: 0.3720117537447143

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.05558385323650008
       0     1     Error    Rate
-----  ----  ----  -------  ---------------
0      7006  2296  0.2468   (2296.0/9302.0)
1      350   325   0.5185   (350.0/675.0)
Total  7356  2621  0.2652   (2646.0/9977.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.0555839    0.197209  185
max f2                       0.0396064    0.336439  255
max f0point5                 0.0849989    0.176556  91
max accuracy                 0.211986     0.932244  0
max precision                0.158951     0.363636  7
max recall                   0.00694898   1         390
max specificity              0.211986     0.999892  0
max absolute_mcc             0.0396064    0.143536  255
max min_per_class_accuracy   0.0477658    0.629972  220
max mean_per_class_accuracy  0.0396064    0.642872  255
max tns                      0.211986     9301      0
max fns                      0.211986     675       0
max fps                      0.00429001   9302      399
max tps                      0.00694898   675       390
max tnr                      0.211986     0.999892  0
max fnr                      0.211986     1         0
max fpr                      0.00429001   1         399
max tpr                      0.00694898   1         390

Gains/Lift Table: Avg response rate:  6.77 %, avg score:  4.06 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100231                   0.116163           3.9908    3.9908             0.27             0.134068    0.27                        0.134068            0.04            0.

In [38]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
DeepLearning_1_AutoML_2_20221113_170232,0.660876,0.232953,0.101359,0.402003,0.242678,0.0588924
GLM_1_AutoML_2_20221113_170232,0.659975,0.227968,0.103387,0.395451,0.242183,0.0586527
GBM_grid_1_AutoML_2_20221113_170232_model_7,0.658466,0.227243,0.103975,0.399014,0.241693,0.0584154
GBM_grid_1_AutoML_2_20221113_170232_model_11,0.6573,0.227401,0.103563,0.403477,0.241711,0.0584244
GBM_grid_1_AutoML_2_20221113_170232_model_14,0.656569,0.227781,0.100379,0.398627,0.241941,0.0585352
GBM_grid_1_AutoML_2_20221113_170232_model_17,0.656038,0.227862,0.102843,0.396492,0.241995,0.0585617
GBM_grid_1_AutoML_2_20221113_170232_model_2,0.656032,0.227613,0.105753,0.402494,0.241728,0.0584322
GBM_grid_1_AutoML_2_20221113_170232_model_24,0.655856,0.227486,0.103301,0.400683,0.241757,0.0584466
GBM_grid_1_AutoML_2_20221113_170232_model_20,0.655556,0.22769,0.102532,0.399894,0.241848,0.0584904
GBM_grid_1_AutoML_2_20221113_170232_model_9,0.654298,0.227761,0.103054,0.3986,0.241821,0.0584775


In [39]:
mdl_automl = h2o.get_model('DeepLearning_1_AutoML_2_20221113_170232')

In [40]:
pred_dt = h2o.as_list(mdl_automl.predict(train_dt_hex),use_pandas=True)
train_dt['score'] = pred_dt['p1']
pred_dt = h2o.as_list(mdl_automl.predict(valid_dt_hex),use_pandas=True)
valid_dt['score'] = pred_dt['p1']

deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


In [56]:
thresh = 0.0549
train_dt['prediction'] = np.where(train_dt.score>=thresh,1,0)
valid_dt['prediction'] = np.where(valid_dt.score>=thresh,1,0)
print(f1_score(y_true=train_dt.is_claim.values,y_pred=train_dt.prediction.values))
print(f1_score(y_true=valid_dt.is_claim.values,y_pred=valid_dt.prediction.values))

0.1891310196394942
0.1776586974443528


In [57]:
pred_dt = h2o.as_list(mdl_automl.predict(test_hex),use_pandas=True)
test['score'] = pred_dt['p1']
test['is_claim'] = np.where(test.score>=thresh,1,0)
print(test[['policy_id','is_claim']].head(5))
test[['policy_id','is_claim']].to_csv('output_iter8.csv',index=None)

deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
  policy_id  is_claim
0   ID58593         1
1   ID58594         0
2   ID58595         0
3   ID58596         0
4   ID58597         0


In [58]:
mdl_automl = h2o.get_model('GLM_1_AutoML_2_20221113_170232')
pred_dt = h2o.as_list(mdl_automl.predict(train_dt_hex),use_pandas=True)
train_dt['score'] = pred_dt['p1']
pred_dt = h2o.as_list(mdl_automl.predict(valid_dt_hex),use_pandas=True)
valid_dt['score'] = pred_dt['p1']

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [75]:
thresh = 0.078
train_dt['prediction'] = np.where(train_dt.score>=thresh,1,0)
valid_dt['prediction'] = np.where(valid_dt.score>=thresh,1,0)
print(f1_score(y_true=train_dt.is_claim.values,y_pred=train_dt.prediction.values))
print(f1_score(y_true=valid_dt.is_claim.values,y_pred=valid_dt.prediction.values))

0.18948187536476233
0.17697729052466718


In [76]:
pred_dt = h2o.as_list(mdl_automl.predict(test_hex),use_pandas=True)
test['score'] = pred_dt['p1']
test['is_claim'] = np.where(test.score>=thresh,1,0)
print(test[['policy_id','is_claim']].head(5))
test[['policy_id','is_claim']].to_csv('output_iter9.csv',index=None)

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
  policy_id  is_claim
0   ID58593         1
1   ID58594         0
2   ID58595         0
3   ID58596         0
4   ID58597         0


In [77]:
thresh = 0.077
train_dt['prediction'] = np.where(train_dt.score>=thresh,1,0)
valid_dt['prediction'] = np.where(valid_dt.score>=thresh,1,0)
print(f1_score(y_true=train_dt.is_claim.values,y_pred=train_dt.prediction.values))
print(f1_score(y_true=valid_dt.is_claim.values,y_pred=valid_dt.prediction.values))

0.19013771657041317
0.1776416539050536


In [ ]:
pred_dt = h2o.as_list(mdl_automl.predict(test_hex),use_pandas=True)
test['score'] = pred_dt['p1']
test['is_claim'] = np.where(test.score>=thresh,1,0)
print(test[['policy_id','is_claim']].head(5))
test[['policy_id','is_claim']].to_csv('output_iter10.csv',index=None)